# 🦜 Wood Pecker Retail Chatbot: Student Guide

**Welcome!** This notebook is a hands-on tutorial for building a retail chatbot. 

You will explore two different ways to build chat logic:
1.  **Manual Rules (The "Reliable" Way):** Using keywords to give exact, pre-written answers.
2.  **NLP Pipeline (The "Smart" Way):** Using AI to understand sentence structure, entities, and sentiment.

By the end, you'll have a working chatbot UI you can talk to!

## 🎯 Learning Goals

*   **Understand Rule-Based Logic:** See how simple keyword matching can solve 80% of customer support queries.
*   **Explore NLP Concepts:** Peek "under the hood" of AI to see how it breaks down text into **Tokens**, **Parts of Speech**, and **Entities**.
*   **Build a UI:** Use the `Gradio` library to launch a real web interface for your bot.

## 🏗️ Project Blueprint

We will build this in **4 steps**:

1.  **The Knowledge Base**: Define a list of common questions (FAQs) and their answers.
2.  **The Brain**: Write a Python class that matches user questions to our FAQs using keywords.
3.  **The NLP Lab**: Set up advanced tools (spaCy, Transformers) to analyze text for sentiment and entities.
4.  **The Interface**: Connect everything to a chat window so we can test it.

In [19]:
# Minimal dependency check: Gradio gives us a quick UI without extra frontend tooling.
import importlib  # Used to check if a library is installed
import subprocess  # Used to run shell commands (like pip install)
import sys  # Used to access system-specific parameters (like the python executable path)


def ensure(package: str) -> None:
    """Install `package` via pip if it is missing."""
    if importlib.util.find_spec(package) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])


ensure("gradio")
ensure("transformers")
ensure("torch")  # PyTorch is the engine behind these Transformers models

import gradio as gr  # Gradio: A library for creating machine learning demos and web UIs quickly
from transformers import pipeline, AutoTokenizer  # Transformers: The library for BERT, GPT, etc.

GREETING_MESSAGE = "Hello, how can I help you today?"

In [20]:
print("Loading BERT models... this may take a minute...")

# --- 1. Tokenizer (BERT Base) ---
# BERT sees text as "sub-words" (tokens). We load the standard tokenizer.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# --- 2. POS Tagging (BERT fine-tuned) ---
# A BERT model fine-tuned to identify Nouns, Verbs, Adjectives, etc.
# Model: vblagoje/bert-english-uncased-finetuned-pos
try:
    pos_pipeline = pipeline("token-classification", model="vblagoje/bert-english-uncased-finetuned-pos")
except Exception as e:
    print(f"Could not load POS model: {e}")
    pos_pipeline = None

# --- 3. NER (BERT fine-tuned) ---
# A BERT model fine-tuned to find Persons, Locations, Organizations (CoNLL-2003 dataset).
# Model: dslim/bert-base-NER
try:
    ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
except Exception as e:
    print(f"Could not load NER model: {e}")
    ner_pipeline = None

# --- 4. Sentiment (DistilBERT) ---
# A lighter BERT model fine-tuned for positive/negative sentiment.
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
)

print("Transformers BERT components loaded successfully!")

Loading BERT models... this may take a minute...


c:\Users\amansahni\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amansahni\.cache\huggingface\hub\models--vblagoje--bert-english-uncased-finetuned-pos. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package

Transformers BERT components loaded successfully!


In [21]:
from datetime import datetime  # Used for handling date and time objects (e.g., timestamps)

faq_items = [
    {
        "question": "What are your store hours?",
        "answer": "We are open 10 AM - 8 PM Monday through Saturday, and 11 AM - 6 PM on Sundays.",
        "keywords": ["hours", "open", "close", "time"],
    },
    {
        "question": "Where is the store located?",
        "answer": "Wood Pecker is inside the Riverwalk Mall, first floor near the fountain.",
        "keywords": ["where", "location", "address", "store"],
    },
    {
        "question": "Do you offer curbside pickup?",
        "answer": "Yes, select curbside pickup at checkout and call us when you arrive at bay C.",
        "keywords": ["curbside", "pickup", "collect"],
    },
    {
        "question": "How can I check my order status?",
        "answer": "Go to woodpecker.com/orders and enter your email plus order number to see live status.",
        "keywords": ["order", "status", "track"],
    },
    {
        "question": "What are the shipping options?",
        "answer": "Standard (5-7 days), Expedited (2-3 days), and Overnight are available at checkout.",
        "keywords": ["shipping", "delivery", "ship"],
    },
    {
        "question": "How long does shipping take?",
        "answer": "Standard shipping arrives within 5-7 business days; peak holidays may add a day.",
        "keywords": ["how long", "shipping", "arrive", "time"],
    },
    {
        "question": "Can I change my shipping address after ordering?",
        "answer": "We can update the address within 30 minutes of purchase—contact support via chat or phone.",
        "keywords": ["change", "address", "order"],
    },
    {
        "question": "Do you have any sales right now?",
        "answer": "Weekly specials post every Friday on our homepage plus 15% off clearance in-store.",
        "keywords": ["sale", "discount", "deal", "promo"],
    },
    {
        "question": "How do returns work?",
        "answer": "Returns are free within 30 days with receipt; bring the item or ship it back with the prepaid label.",
        "keywords": ["return", "policy", "refund"],
    },
    {
        "question": "Can I exchange a gift?",
        "answer": "Absolutely—bring the gift receipt or sender name within 60 days for an exchange or store credit.",
        "keywords": ["exchange", "gift"],
    },
    {
        "question": "Do you sell gift cards?",
        "answer": "Gift cards are available from $10 to $500 both online and at the register.",
        "keywords": ["gift", "card", "voucher"],
    },
    {
        "question": "How do I apply a promo code?",
        "answer": "Enter the code in the Promo Code box at checkout; it will show the discount instantly.",
        "keywords": ["promo", "code", "coupon"],
    },
    {
        "question": "What sizes do you carry?",
        "answer": "Most apparel ranges from XS-3XL, and footwear spans sizes 5-13 for adults.",
        "keywords": ["size", "sizing", "fit"],
    },
    {
        "question": "Is there a loyalty program?",
        "answer": "Yes—Woodland Rewards gives 1 point per dollar and $5 back for every 100 points.",
        "keywords": ["loyalty", "membership", "rewards"],
    },
    {
        "question": "Can I talk to a human agent?",
        "answer": "Sure thing. Call 800-555-7425 or use the Help > Live Chat option for a teammate.",
        "keywords": ["agent", "human", "representative", "contact"],
    },
    {
        "question": "Do you offer repairs or alterations?",
        "answer": "Basic hemming and zipper repairs are available for $15—drop items at the service desk.",
        "keywords": ["repair", "alteration", "tailor"],
    },
    {
        "question": "How do I cancel an order?",
        "answer": "You can cancel within 30 minutes via your order page or by calling customer care.",
        "keywords": ["cancel", "order"],
    },
    {
        "question": "Are holiday items in stock?",
        "answer": "Seasonal decor is refreshed every Thursday; check the Holiday tab for live inventory badges.",
        "keywords": ["holiday", "seasonal", "stock"],
    },
    {
        "question": "Do you price match?",
        "answer": "Yes, we match major retailers within 14 days of purchase—bring the ad or link.",
        "keywords": ["price match", "match", "price"],
    },
    {
        "question": "What payment methods are accepted?",
        "answer": "We accept major cards, PayPal, Apple Pay, Google Pay, and contactless in-store.",
        "keywords": ["payment", "pay", "methods"],
    },
    {
        "question": "How do I get notified about new products?",
        "answer": "Subscribe to our newsletter or enable push alerts in the Wood Pecker mobile app.",
        "keywords": ["notify", "new", "products", "newsletter"],
    },
    {
        "question": "Can I buy online and return in store?",
        "answer": "Yes, bring the packing slip or email receipt to any Wood Pecker location.",
        "keywords": ["buy online", "return", "store"],
    },
    {
        "question": "Is same-day delivery available?",
        "answer": "Same-day courier delivery is available within 15 miles for $12 on orders placed before 2 PM.",
        "keywords": ["same day", "delivery", "courier"],
    },
    {
        "question": "Do you have eco-friendly packaging?",
        "answer": "We default to recyclable packaging and offer a $1 rebate when you opt for minimal packing.",
        "keywords": ["eco", "sustainable", "packaging"],
    },
    {
        "question": "Can I reserve items to try later?",
        "answer": "Use the Reserve & Try feature online to hold items for 24 hours at your chosen store.",
        "keywords": ["reserve", "hold", "try"],
    },
    {
        "question": "How do I contact support after hours?",
        "answer": "Leave us a voicemail at 800-555-7425 or email support@woodpecker.com—we reply next morning.",
        "keywords": ["support", "after hours", "contact"],
    },
    {
        "question": "Where can I see my loyalty balance?",
        "answer": "Log in to your Wood Pecker account and look for the Rewards tab for real-time points.",
        "keywords": ["loyalty", "points", "balance"],
    },
    {
        "question": "What is Wood Pecker's return address?",
        "answer": "Ship returns to Wood Pecker Returns, 422 Orchard Lane, Columbus, OH 43215.",
        "keywords": ["return", "address", "ship back"],
    },
    {
        "question": "Do you assemble furniture?",
        "answer": "We partner with HandyCo for in-home assembly starting at $49; schedule at checkout.",
        "keywords": ["assemble", "assembly", "furniture"],
    },
]

## 🧩 Understanding the Code Components

Before we run the logic, here is a quick guide to the key parts you are about to see:

*   **`faq_items`**: A simple list of dictionaries. This is our bot's "memory".
*   **`SimpleRetailBrain`**: The decision maker. It looks at the user's message, counts how many keywords match an FAQ, and picks the winner.
*   **`sentiment_analyzer`**: A pre-trained AI model (from Hugging Face) that detects if a user is happy or frustrated.
*   **`analyze_text_simple`**: A teaching function. It takes a sentence and prints out all the NLP data (tokens, nouns, entities) so you can see what the computer "sees".

### 💡 Concept Spotlight: BERT Named Entity Recognition (NER)

In the **NLP Lab**, we now use a **BERT** model trained on the CoNLL-2003 dataset. 

Unlike the previous spaCy model, this BERT model focuses on four main entity types:
*   **PER**: Person (e.g., "Serena Williams")
*   **ORG**: Organization (e.g., "Wood Pecker", "Google")
*   **LOC**: Location (e.g., "Paris", "Riverwalk Mall")
*   **MISC**: Miscellaneous (e.g., "German", "Super Bowl")

*Note: BERT uses "sub-word" tokenization, so you might see tokens like `##ing` or `##s` in the analysis output!*

In [22]:
from typing import Callable, Dict, List, Optional, Tuple  # Typing: Provides support for type hints to improve code readability and debugging


class SimpleRetailBrain:
    """Transparent keyword-based router with an optional ML override hook."""

    def __init__(self, faqs: List[Dict[str, List[str]]]):
        self.faqs = faqs
        self.ml_handler: Optional[Callable[[str], Optional[str]]] = None

    def set_ml_handler(self, handler: Callable[[str], Optional[str]]) -> None:
        """Allow future ML models (LLMs, vector search, etc.) to override answers."""
        self.ml_handler = handler

    def _score(self, normalized_prompt: str, keywords: List[str]) -> int:
        return sum(1 for keyword in keywords if keyword in normalized_prompt)

    def answer(self, prompt: str) -> Tuple[str, str]:
        normalized = prompt.lower().strip()
        if self.ml_handler:  # plug in semantic search or an LLM later
            ml_answer = self.ml_handler(prompt)
            if ml_answer:
                return ml_answer, "ml_handler"

        best_answer = None
        best_question = None
        best_score = 0
        for item in self.faqs:
            score = self._score(normalized, item["keywords"])
            if score > best_score:
                best_score = score
                best_answer = item["answer"]
                best_question = item["question"]

        if best_answer is None:
            fallback = (
                "I do not have that in my notes yet. Could you try rephrasing or "
                "contact our team at support@woodpecker.com?"
            )
            return fallback, "fallback"

        return best_answer, best_question or "faq"


brain = SimpleRetailBrain(faq_items)

In [23]:
from collections import defaultdict
from datetime import datetime

# --- 2. Helper Functions ---

def format_answer(answer: str, source: str) -> str:
    """Adds a timestamp and source label to the bot's reply."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    return f"[{timestamp}] ({source}) {answer}"


def summarize_entities(ner_results) -> Dict[str, List[str]]:
    """Group detected entities by their BERT labels (PER, ORG, LOC, etc.)."""
    if not ner_results:
        return {}
        
    summary: Dict[str, List[str]] = defaultdict(list)
    for ent in ner_results:
        # Transformers pipeline with aggregation_strategy='simple' returns 'entity_group'
        label = ent['entity_group']
        text = ent['word']
        summary[label].append(text)
    
    # Deduplicate while preserving order for readability
    return {label: sorted(set(values), key=values.index) for label, values in summary.items()}


def analyze_text_simple(text: str) -> str:
    """
    Runs a pure Transformers (BERT) pipeline.
    Steps: BERT Tokenizer -> BERT POS -> BERT NER -> BERT Sentiment
    """
    if not text:
        return "Please enter text to analyze."

    # 1. Tokenization (BERT Tokenizer)
    # We use the tokenizer directly to show the sub-word tokens
    tokens = tokenizer.tokenize(text)

    # 2. POS Tagging (BERT Model)
    if pos_pipeline:
        pos_data = pos_pipeline(text)
        # Format: "word (TAG)" - BERT POS models often return tags like 'NOUN', 'VERB'
        readable_pos = [f"{item['word']} ({item['entity']})" for item in pos_data]
    else:
        readable_pos = ["(POS Model not loaded)"]

    # 3. NER (BERT Model)
    if ner_pipeline:
        ner_data = ner_pipeline(text)
        entities_by_label = summarize_entities(ner_data)
    else:
        entities_by_label = "(NER Model not loaded)"

    # 4. Sentiment (DistilBERT)
    sentiment = sentiment_analyzer(text)[0]

    return (
        f"### Analysis of: '{text}'\n\n"
        f"**1. BERT Tokens:** {tokens}\n\n"
        f"**2. POS Tags:** {', '.join(readable_pos)}\n\n"
        f"**3. Entities (BERT NER):** {entities_by_label or 'None'}\n\n"
        f"**4. Sentiment:** {sentiment['label']} ({sentiment['score']:.2f})"
    )

# --- 3. Chatbot Logic ---

def respond(message: str, history: List[Dict[str, str]], mode: str):
    """Main function called by Gradio when user sends a message."""
    history = history or [{"role": "assistant", "content": GREETING_MESSAGE}]

    if mode == "Sentiment pipeline":
        result = sentiment_analyzer(message)[0]
        reply = f"Sentiment: {result['label']} (Score: {result['score']:.2f})"
        source = "sentiment-model"
    else:
        reply, source = brain.answer(message)

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": format_answer(reply, source)})
    return history, ""


def reset_chat():
    return [{"role": "assistant", "content": GREETING_MESSAGE}], ""

# --- 4. UI Configuration ---
CUSTOM_CSS = """
#chatbot .message.user { background-color: #fff4cf; color: #663c00; }
#chatbot .message.bot { background-color: #dceeff; color: #003355; }
"""
STYLE_TAG = f"<style>{CUSTOM_CSS}</style>"

In [24]:
with gr.Blocks() as demo:
    gr.HTML(STYLE_TAG)
    gr.Markdown("""
**How to use this sandbox**
1. **Chat**: Ask Wood Pecker about orders, returns, promos, shipping, hours, etc.
2. **Modes**: Choose **Manual data (FAQs)** for rule-based answers or **Sentiment pipeline** for the transformers option.
3. **NLP Practice**: Use the **User feedback** box on the right to test the integrated NLP pipeline (Tokenization -> POS -> NER -> Sentiment). Try a sentence like: *"Serena Williams won Wimbledon in 2016."*
""")
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### Chat with Wood Pecker")
            chatbot = gr.Chatbot(
                label="Wood Pecker",
                height=420,
                elem_id="chatbot",
                value=[{"role": "assistant", "content": GREETING_MESSAGE}],
            )
            with gr.Row():
                msg = gr.Textbox(
                    label="Ask a question",
                    placeholder="e.g., How fast is shipping?",
                    elem_classes=["input-box"],
                    scale=3,
                )
                mode = gr.Radio(
                    choices=[MODE_MANUAL, MODE_SENTIMENT],
                    value=MODE_MANUAL,
                    label="Answer mode",
                    info="Option 1 = manual data, Option 2 = transformers sentiment pipeline.",
                    scale=1,
                )
            msg.submit(respond, inputs=[msg, chatbot, mode], outputs=[chatbot, msg])
            clear = gr.Button("Clear conversation")
            clear.click(reset_chat, None, [chatbot, msg], queue=False)
        with gr.Column(scale=1):
            gr.Markdown("### User feedback / NLP Lab")
            feedback_input = gr.Textbox(
                label="Share your experience (or test text)",
                placeholder="Type any text here to analyze...",
                lines=4,
                elem_classes=["input-box"],
            )
            feedback_button = gr.Button("Analyze NLP Pipeline")
            feedback_result = gr.Markdown(value="Awaiting input...")
            feedback_button.click(analyze_feedback, inputs=feedback_input, outputs=feedback_result)
    gr.Markdown("""
_Tip_: to test a future ML model, pass a callable to `brain.set_ml_handler` that takes the prompt and returns a custom string. Leave it `None` to stay purely rule-based.
""")

# Launch on any available port to avoid conflicts when rerunning in notebooks.
demo.launch(server_name="0.0.0.0")

ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7862): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7862): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted


* Running on local URL:  http://0.0.0.0:7863
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.


## 🚀 Next Steps: Making it Smarter

Once you understand the basics, here is how you could upgrade this bot:

*   **Semantic Search**: Instead of matching keywords ("shipping"), use vectors to match meaning ("how do I get my stuff?").
*   **API Integration**: Connect the "Order Status" answer to a real database to give live updates.
*   **Context Awareness**: Make the bot remember your name or previous questions.

In [24]:
# Safety net: close any previously running Gradio demo before relaunching.
try:
    demo.close()
except NameError:
    pass